In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("GEOquery", ask = FALSE)
BiocManager::install("limma", ask = FALSE)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.25), R 4.5.0 (2025-04-11)

Installing package(s) 'BiocVersion', 'GEOquery'

also installing the dependencies ‘matrixStats’, ‘XVector’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘abind’, ‘SparseArray’, ‘BiocGenerics’, ‘statmod’, ‘XML’, ‘R.oo’, ‘R.methodsS3’, ‘MatrixGenerics’, ‘GenomicRanges’, ‘IRanges’, ‘GenomeInfoDb’, ‘S4Arrays’, ‘DelayedArray’, ‘Biobase’, ‘limma’, ‘rentrez’, ‘R.utils’, ‘SummarizedExperiment’, ‘S4Vectors’


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.25), R 4.5.0 (2025-04-1

In [ ]:
library(GEOquery)
library(limma)


Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: generics


Attaching package: ‘generics’


The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union



Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, saveRDS, table, tapply, unique,
    unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Bioba

In [ ]:
install.packages("umap")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’, ‘RcppEigen’, ‘reticulate’, ‘RSpectra’




In [ ]:
library(umap)


In [ ]:
# Version info: R 4.2.2, Biobase 2.58.0, GEOquery 2.66.0, limma 3.54.0
################################################################
#   Differential expression analysis with limma
library(GEOquery)
library(limma)
library(umap)

# load series and platform data from GEO

gset <- getGEO("GSE45827", GSEMatrix =TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep("GPL570", attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

# make proper column names to match toptable
fvarLabels(gset) <- make.names(fvarLabels(gset))

# group membership for all samples
gsms <- paste0("11111111111111111111111111111111111111111111111XXX",
        "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX0000000000000XXXX",
        "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
        "XXXXX")
sml <- strsplit(gsms, split="")[[1]]

# filter out excluded samples (marked as "X")
sel <- which(sml != "X")
sml <- sml[sel]
gset <- gset[ ,sel]

# log2 transformation
ex <- exprs(gset)
qx <- as.numeric(quantile(ex, c(0., 0.25, 0.5, 0.75, 0.99, 1.0), na.rm=T))
LogC <- (qx[5] > 100) ||
          (qx[6]-qx[1] > 50 && qx[2] > 0)
if (LogC) { ex[which(ex <= 0)] <- NaN
  exprs(gset) <- log2(ex) }

# assign samples to groups and set up design matrix
gs <- factor(sml)
groups <- make.names(c("Normal Controls","Tumour Controls"))
levels(gs) <- groups
gset$group <- gs
design <- model.matrix(~group + 0, gset)
colnames(design) <- levels(gs)

gset <- gset[complete.cases(exprs(gset)), ] # skip missing values

fit <- lmFit(gset, design)  # fit linear model

# set up contrasts of interest and recalculate model coefficients
cts <- paste(groups[1], groups[2], sep="-")
cont.matrix <- makeContrasts(contrasts=cts, levels=design)
fit2 <- contrasts.fit(fit, cont.matrix)

# compute statistics and table of top significant genes
fit2 <- eBayes(fit2, 0.01)
tT <- topTable(fit2, adjust="fdr", sort.by="B", number=250)

Found 1 file(s)

GSE45827_series_matrix.txt.gz

Using locally cached version: /tmp/RtmplgehwL/GSE45827_series_matrix.txt.gz

Using locally cached version of GPL570 found here:
/tmp/RtmplgehwL/GPL570.annot.gz 



In [ ]:
# Differential expression analysis with limma
library(GEOquery)
library(limma)
library(umap)

# Load series and platform data from GEO
gset <- getGEO("GSE45827", GSEMatrix = TRUE, AnnotGPL = TRUE)
if (length(gset) > 1) idx <- grep("GPL570", attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

# Make proper column names
fvarLabels(gset) <- make.names(fvarLabels(gset))

# Group membership for all samples
gsms <- paste0("11111111111111111111111111111111111111111111111XXX",
               "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX0000000000000XXXX",
               "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
               "XXXXX")
sml <- strsplit(gsms, split = "")[[1]]

# Filter out excluded samples (marked as "X")
sel <- which(sml != "X")
sml <- sml[sel]
gset <- gset[, sel]

# Log2 transformation if necessary
ex <- exprs(gset)
qx <- as.numeric(quantile(ex, c(0., 0.25, 0.5, 0.75, 0.99, 1.0), na.rm = TRUE))
LogC <- (qx[5] > 100) || (qx[6] - qx[1] > 50 && qx[2] > 0)
if (LogC) {
  ex[which(ex <= 0)] <- NaN
  exprs(gset) <- log2(ex)
}

# Assign samples to groups and set up design matrix
gs <- factor(sml)
groups <- make.names(c("Normal Controls", "Tumour Controls"))
levels(gs) <- groups
gset$group <- gs
design <- model.matrix(~ group + 0, gset)
colnames(design) <- levels(gs)

# Remove missing values
gset <- gset[complete.cases(exprs(gset)), ]

# Fit linear model
fit <- lmFit(gset, design)

# Set up contrasts
cts <- paste(groups[1], groups[2], sep = "-")
cont.matrix <- makeContrasts(contrasts = cts, levels = design)
fit2 <- contrasts.fit(fit, cont.matrix)
fit2 <- eBayes(fit2, 0.01)

# Top table
tT <- topTable(fit2, adjust = "fdr", sort.by = "B", number = 250)
tT <- tT$ID <- rownames(tT)
annotated_tT <- merge(tT, fData(gset), by = "ID")
subset(annotated_tT, select = c("ID", "logFC", "P.Value", "adj.P.Val", "Gene.symbol", "Gene.title"))

write.table(tT, file = stdout(), row.names = FALSE, sep = "\t")

# Histogram of adjusted P-values
tT2 <- topTable(fit2, adjust = "fdr", sort.by = "B", number = Inf)
hist(tT2$adj.P.Val, col = "grey", border = "white",
     xlab = "Adjusted P-value", ylab = "Number of genes",
     main = "P-adj value distribution")

# Decide test results
dT <- decideTests(fit2, adjust.method = "fdr", p.value = 0.05, lfc = 0)

# Venn diagram
vennDiagram(dT, circle.col = palette())

# Q-Q plot
t.good <- which(!is.na(fit2$F))
qqt(fit2$t[t.good], fit2$df.total[t.good], main = "Moderated t statistic")

# Volcano plot
colnames(fit2)
ct <- 1
volcanoplot(fit2, coef = ct, main = colnames(fit2)[ct], pch = 20,
            highlight = length(which(dT[, ct] != 0)), names = rep('+', nrow(fit2)))

# MD plot
plotMD(fit2, column = ct, status = dT[, ct], legend = FALSE, pch = 20, cex = 1)
abline(h = 0)

# General expression plots
ex <- exprs(gset)

# Boxplot
dev.new(width = 3 + ncol(gset) / 6, height = 5)
ord <- order(gs)
palette(c("#1B9E77", "#7570B3", "#E7298A", "#E6AB02", "#D95F02",
          "#66A61E", "#A6761D", "#B32424", "#B324B3", "#666666"))
par(mar = c(7, 4, 2, 1))
boxplot(ex[, ord], boxwex = 0.6, notch = TRUE,
        main = paste("GSE45827 /", annotation(gset)),
        outline = FALSE, las = 2, col = gs[ord])
legend("topleft", groups, fill = palette(), bty = "n")
dev.off()

# Density plot
par(mar = c(4, 4, 2, 1))
plotDensities(ex, group = gs, main = paste("GSE45827 /", annotation(gset), "value distribution"), legend = "topright")

# UMAP plot
ex <- na.omit(ex)
ex <- ex[!duplicated(ex), ]
ump <- umap(t(ex), n_neighbors = 15, random_state = 123)
par(mar = c(3, 3, 2, 6), xpd = TRUE)
plot(ump$layout, main = "UMAP plot, nbrs=15", xlab = "", ylab = "", col = gs, pch = 20, cex = 1.5)
legend("topright", inset = c(-0.15, 0), legend = levels(gs), pch = 20,
       col = 1:nlevels(gs), title = "Group", pt.cex = 1.5)

# Mean-variance trend plot
plotSA(fit2, main = "Mean variance trend, GSE45827")


Found 1 file(s)

GSE45827_series_matrix.txt.gz

Using locally cached version: /tmp/RtmplgehwL/GSE45827_series_matrix.txt.gz

Using locally cached version of GPL570 found here:
/tmp/RtmplgehwL/GPL570.annot.gz 



ERROR: Error in fix.by(by.x, x): 'by' must specify a uniquely valid column
